# RAG 증강, 생성
- Retrieval, Augmented, Generation

## 환경변수 로딩

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

## 필요한 라이브러리 로딩

In [2]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

## 검색

### 질문에 대한 Top k 겁색

In [3]:
# 전체 코드 통합, 함수로 정의
question = "김대중 대통령의 행적에 대해 알려줘"



def sm_search_top_k(question):
    
    #벡터db, index 객체 생성
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "wiki"
    index = pc.Index(index_name)
    
    # embedding 객체 생성
    embedding = OpenAIEmbeddings(
        model = "text-embedding-3-small",
        api_key = OPENAI_API_KEY)
    
    
    # 질문을 embedding vector로 변경  <---  results = vector_store.similarity_search() 여기서 자동으로 됨
    # embedded_question = embedding.embed_documents(question)
    
    

    # top 3 검색
    # 텍스트 키는 "chunk_text"로 설정합니다.
    vector_store = PineconeVectorStore(index=index,        
                                       embedding=embedding, 
                                       text_key="chunk_text")  # default text 임, 우리가 설정한 값이 chunk_text
    
    # 질문에 대한 유사도 검색을 수행합니다.
    # query는 question 리스트의 첫 번째 요소를 사용하고, 상위 3개의 결과를 반환합니다.
    results = vector_store.similarity_search(
        query=question, # question 문자열을 embedding vector로 자동 변환됨
        k=3, 
        namespace="wiki_ns1")
    # print(results)
    
    # 검색된 문서의 메타데이터를 출력합니다.
    context = ""
    for result in results:
        # print(result)
        # print(result.metadata)
        # print(result.page_content)
        # print("-"*30)
        context = context + result.page_content
        
    return context

In [4]:
# 함수 호출
question = "김대중 대통령의 행적에 대해 알려줘"
top_k_context = sm_search_top_k(question)
print(top_k_context)

취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아1995년 부산 시장 선거 출마, 통합민주당 창당 활동에 합류
 1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령아니라, 그동안 여러분에게 지고 있었던 마음의 짐을 풀었다는 것이 가장 기쁩니다"라는 말로 과거의 일을 반성했다. 그리고 그 해 12월 18일, 김대중이 15대 대통령에 당선되면서 노무현은 사상 처음으로 여당에 몸담게 되었다.


## 증강

### 프롬프트 템플릿 만들기

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 템플릿 구성
chat_template = ChatPromptTemplate([
    # 시스템 프롬프트
    ("system", "당신은 역사전문가 특히 인문에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요."),
    # user 프롬프트
    ("human", """{question}에 대해 설명해 주세요.
                 아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.
                 context:{context}""")]  
)
chat_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 역사전문가 특히 인문에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='{question}에 대해 설명해 주세요.\n                 아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.\n                 context:{context}'), additional_kwargs={})])

### 질문 만들기

In [10]:
# 질문
question = "백남준의 예술 세계에 대해 설명해줘"
# question = "링컨 대통령에 대해 설명해주세요."

### 질문에 대한 Top-k 검색

In [11]:
top_k_context = sm_search_top_k(question)
print(top_k_context)

각주

외부 링크 

  백남준아트센터
 ARTCYCLOPEDIA에서 백남준 항목
 백남준의 세계 (출처:Guggenheim.org)
 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)
 백남준 일대기 (출처:@ MedienKunstNetz)}
 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)
 백남준 부고 "Video artist Nam June Paik dead at 74", (출처:뉴욕 타임즈, 2006년 1월 30일 보도)
 백남준 공식 블로그관련 영상 
 백남준의 위성예술 1부 (1984년 방송)
 백남준의 위성예술 2부 (1984년 방송)
 백남준의 비디오아트세계 (1984년 방송)
 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집

각주

외부 링크백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).

생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 '비디오 아트의 창시자'로 알려져 있다.

생애


### 최종 프롬프트 만들기
- 템플릿프롬프트 + 사용자질문 + top-k context

In [12]:
prompt = chat_template.format_messages(
    question =question,
    context = top_k_context
)
prompt

[SystemMessage(content='당신은 역사전문가 특히 인문에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='백남준의 예술 세계에 대해 설명해줘에 대해 설명해 주세요.\n                 아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.\n                 context:각주\n\n외부 링크 \n\n  백남준아트센터\n ARTCYCLOPEDIA에서 백남준 항목\n 백남준의 세계 (출처:Guggenheim.org)\n 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)\n 백남준 일대기 (출처:@ MedienKunstNetz)}\n 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)\n 백남준 부고 "Video artist Nam June Paik dead at 74", (출처:뉴욕 타임즈, 2006년 1월 30일 보도)\n 백남준 공식 블로그관련 영상 \n 백남준의 위성예술 1부 (1984년 방송)\n 백남준의 위성예술 2부 (1984년 방송)\n 백남준의 비디오아트세계 (1984년 방송)\n 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집\n\n각주\n\n외부 링크백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).\n\n생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 \'비디오 아트의 창시자\'로 알려져 있다.\n\n생애', add

## 생성
- LLM이 프롬프트에 따라 결과를 생성함

In [13]:
# openai gpt 모델 사용
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [14]:
response = llm.invoke(prompt)

In [15]:
response.content

'백남준(백남준, 1932년 7월 20일 ~ 2006년 1월 29일)은 한국 태생의 세계적인 비디오 아트 예술가이자 작곡가, 전위 예술가입니다. 그는 서울에서 태어나 생전에 뉴욕, 쾰른, 도쿄, 마이애미 그리고 서울에서 주로 거주하며 다양한 매체로 예술 활동을 하였습니다.\n\n백남준은 특히 비디오 아트를 새로운 예술의 범주로 발전시킨 것으로 평가받아 "비디오 아트의 창시자"로 알려져 있습니다. 그의 예술 세계는 혁신적이고 실험적인 접근을 통해 예술의 경계를 확장하며, 현대 미술에 중요한 영향을 끼쳤습니다. 이러한 맥락에서 백남준은 단순히 비디오 매체의 사용을 넘어서, 기술과 예술, 문화의 융합을 탐구하는 데 중점을 두었습니다. \n\n그의 작품은 일반적으로 비디오 설치물, 인터미디어 작업 등으로 구성되어 있으며, 사람들이 비디오 매체를 통해 새로운 경험을 할 수 있도록 유도합니다. 백남준의 작품은 현대 사회와 기술의 변화를 반영하며, 예술의 의미와 형식을 재조명하는 데 중점을 두었습니다.'

# LCEL로 chain을 만들어서 실행하기

In [17]:
# 실행 파이프라인 구성
chain = prompt | llm

# 프롬프트템플릿의 변수에 대입할 값
input_data = {"question": question ,"context": top_k_context}

# chain 실행
response = chain.invoke(input_data)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>

In [ ]:
result = response.content